In [1]:
%load_ext autoreload
%autoreload 2
from cdf_auth import get_client_demo # just a function to get a CogniteClient
from cognite import pygen # pip install cognite-pygen
from cognite.powerops import PowerOpsClient

client = get_client_demo()
po_client = PowerOpsClient(read_dataset="powerops:misc", write_dataset="powerops:misc", cogshop_version="0.0.1", config=client.config)

print(pygen.__version__)

0.99.24


In [2]:
#space = "benchmarking_sp_power_ops_models"
#data_model_external_id = "compute_DayAheadBenchmarking"
space = "final_thomi"
data_model_external_id = "compute_BenchmarkingDayAhead"
data_model_version = "1"
data_model_id = (space, data_model_external_id, data_model_version)
bench_client = pygen.generate_sdk_notebook(data_model_id, client)

Successfully retrieved data model(s) ('final_thomi', 'compute_BenchmarkingDayAhead', '1')
Cleaned temporary directory /tmp/pygen/final_thomi_compute_BenchmarkingDayAhead_1


/home/thomas/virtualenv/cognite-power-ops-dev/lib/python3.11/site-packages/cognite/pygen/_core/generators.py:305: UserWarning: Field(s) name in view ViewId(space='final_thomi', external_id='ShopModel', version='1') has potential conflict with protected Pydantic namespace 'model_'
  sdk[data_classes_dir / f"_{file_name}.py"] = api.generate_data_class_file(self.pydantic_version == "v2")
/home/thomas/virtualenv/cognite-power-ops-dev/lib/python3.11/site-packages/cognite/pygen/_core/generators.py:305: UserWarning: Field(s) name in view ViewId(space='final_thomi', external_id='ShopScenario', version='1') has potential conflict with protected Pydantic namespace 'model_'
  sdk[data_classes_dir / f"_{file_name}.py"] = api.generate_data_class_file(self.pydantic_version == "v2")


Writing SDK to /tmp/pygen/final_thomi_compute_BenchmarkingDayAhead_1
Done!
Added /tmp/pygen/final_thomi_compute_BenchmarkingDayAhead_1 to sys.path to enable import
Imported compute_benchmarking_day_ahead
You can now use the generated SDK in the current Python session.
The data classes are available by importing, for example, `from compute_benchmarking_day_ahead.data_classes import BenchmarkingConfigurationDayAheadWrite`


In [3]:
import compute_benchmarking_day_ahead.data_classes as data_classes

In [4]:
#po_client.v1.day_ahead_configuration.bid_configuration_day_ahead.list()
bid_config_ext_ids = ["bid_configuration_day_ahead_multi_scenario_2_no2", "bid_configuration_day_ahead_water_value_no2_full", "bid_configuration_day_ahead_no2_combination", "bid_configuration_day_ahead_price_independent_no2"] 
bid_configs = po_client.v1.day_ahead_configuration.bid_configuration_day_ahead.retrieve(external_id=bid_config_ext_ids)
# shop_models = po_client.v1.shop_based_day_ahead_bid_process.shop_model.list(space="sp_power_ops_instance", limit=1000)
from cognite.powerops.client._generated.v1.data_classes import BidConfigurationDayAheadWrite

shop_model_dummy_1 = data_classes.ShopModelWrite(
    externalId="shop_model_benchmarking_dummy_1",
    name="Fornebu B",
    shopVersion="15.4.1",
    model="SHOP_Fornebu_model"
)
shop_model_dummy_2 = data_classes.ShopModelWrite(
    externalId="shop_model_benchmarking_dummy_2",
    name="Lysakerelva B",
    shopVersion="15.4.1",
    model="SHOP_Fornebu_model"
)
shop_scenario_dummy_1 = data_classes.ShopScenarioWrite(
    externalId="shop_scenario_benchmarking_dummy_1",
    name="Fornebu B Benchmarking",
    model=shop_model_dummy_1
)
shop_scenario_dummy_2 = data_classes.ShopScenarioWrite(
    externalId="shop_scenario_benchmarking_dummy_2",
    name="Lysakerelva B Benchmarking",
    model=shop_model_dummy_2,
)
shop_case_dummy_1 = data_classes.ShopCaseWrite(
    externalId="shop_case_benchmarking_dummy_1",
    name="Lysakerelva B Benchmarking dummy",
    scenario=shop_scenario_dummy_1
)
shop_case_dummy_2 = data_classes.ShopCaseWrite(
    externalId="shop_case_benchmarking_dummy_2",
    name="Lysakerelva B Benchmarking dummy",
    scenario=shop_scenario_dummy_2
)
shop_result_dummy_1 = data_classes.ShopResultWrite(
    externalId="shop_result_benchmarking_dummy_1",
    name="Fornebu B Benchmarking dummy",
    case=shop_case_dummy_1
)
shop_result_dummy_2 = data_classes.ShopResultWrite(
    externalId="shop_result_benchmarking_dummy_2",
    name="Lysakerelva B Benchmarking dummy",
    case=shop_case_dummy_2,
)

shop_results_upserted = bench_client.upsert([shop_result_dummy_1, shop_result_dummy_2])

bid_sources = [
    BidConfigurationDayAheadWrite(space="sp_power_ops_model_benchmarking", externalId=bid_config.external_id, name=bid_config.name, bidDateSpecification="[]")
    for bid_config in bid_configs] + []

bid_sources = [bid_config.external_id for bid_config in bid_configs] + ["upper_bound"]

#bid_documents_upserted = bench_client.upsert(bid_sources)

In [5]:
from pprint import pprint
pprint(shop_results_upserted)

ResourcesWriteResult(nodes=[<NodeApplyResult(space='final_thomi', external_id='shop_result_benchmarking_dummy_1', version=1) at 0x7f780458c450>,
                            <NodeApplyResult(space='final_thomi', external_id='shop_case_benchmarking_dummy_1', version=2) at 0x7f780458d490>,
                            <NodeApplyResult(space='final_thomi', external_id='shop_scenario_benchmarking_dummy_1', version=2) at 0x7f780458d150>,
                            <NodeApplyResult(space='final_thomi', external_id='shop_model_benchmarking_dummy_1', version=2) at 0x7f780458d3d0>,
                            <NodeApplyResult(space='final_thomi', external_id='shop_result_benchmarking_dummy_2', version=1) at 0x7f780458d310>,
                            <NodeApplyResult(space='final_thomi', external_id='shop_case_benchmarking_dummy_2', version=2) at 0x7f780458c290>,
                            <NodeApplyResult(space='final_thomi', external_id='shop_scenario_benchmarking_dummy_2', version=2) at 0x7

In [6]:
import datetime
import random

def get_value(bid_source_external_id: str, delivery_date: str, model_external_id: str) -> float:
    if bid_source_external_id == "upper_bound":
        return 0
    return random.random()

results = []
delivery_dates = [datetime.date(2024, 5, day) for day in range(1, 32)]
dummy_shop_results = [shop_result_dummy_1, shop_result_dummy_2]

for bid_source in bid_sources:
    for delivery_date in delivery_dates:
        for dummy_shop_result in dummy_shop_results:

            results.append(
                data_classes.BenchmarkingResultDayAheadWrite(
                    externalId = f"benchmarking_result_day_ahead_{dummy_shop_result.case.scenario.model.external_id}_{bid_source}_{delivery_date}_1",
                    bidSource = bid_source,
                    deliveryDate = delivery_date,
                    bidGenerated = None,
                    shopResult = dummy_shop_result.external_id,
                    isSelected = True,
                    value = get_value(bid_source, delivery_date, dummy_shop_result.case.scenario.model.external_id)
            ))
bench_client.upsert(results)

ResourcesWriteResult(nodes=[<NodeApplyResult(space='final_thomi', external_id='benchmarking_result_day_ahead_shop_model_benchmarking_dummy_1_bid_configuration_day_ahead_multi_scenario_2_no2_2024-05-01_1', version=1) at 0x7f78046b9590>, <NodeApplyResult(space='final_thomi', external_id='benchmarking_result_day_ahead_shop_model_benchmarking_dummy_2_bid_configuration_day_ahead_multi_scenario_2_no2_2024-05-01_1', version=1) at 0x7f77ff26be90>, <NodeApplyResult(space='final_thomi', external_id='benchmarking_result_day_ahead_shop_model_benchmarking_dummy_1_bid_configuration_day_ahead_multi_scenario_2_no2_2024-05-02_1', version=1) at 0x7f78058faa90>, <NodeApplyResult(space='final_thomi', external_id='benchmarking_result_day_ahead_shop_model_benchmarking_dummy_2_bid_configuration_day_ahead_multi_scenario_2_no2_2024-05-02_1', version=1) at 0x7f77ff26bf90>, <NodeApplyResult(space='final_thomi', external_id='benchmarking_result_day_ahead_shop_model_benchmarking_dummy_1_bid_configuration_day_ahead

In [7]:
bm_config = data_classes.BenchmarkingConfigurationDayAheadWrite(
    externalId="day_ahead_benchmarking_configuration_no2",
    name="Day Ahead Benchmarking Configuration Dummy",
    bidConfigurations=bid_config_ext_ids,
    shopStartSpecification="[]",
    shopEndSpecification="[]",
    assetsPerShopModel=[data_classes.ShopModelWithAssetsWrite(
        externalId=f"shop_model_to_assets_dummy_{shop_model.external_id[-1:]}",
        shopModel=shop_model.external_id,
        powerAssets=[],
        productionObligations=[],
    ) for shop_model in [shop_model_dummy_1, shop_model_dummy_2]],

)
upserted_bm_config = bench_client.upsert(bm_config)

In [9]:
bench_client.benchmarking_result_day_ahead.list()

,space,external_id,name,workflow_execution_id,bid_source,delivery_date,bid_generated,shop_result,is_selected,value,alerts,node_type,data_record
0,final_thomi,benchmarking_result_day_ahead_shop_model_bench...,None,None,bid_configuration_day_ahead_multi_scenario_2_no2,2024-05-01,None,shop_result_benchmarking_dummy_1,True,0.635710,None,"{'space': 'final_thomi', 'external_id': 'Bench...","{'version': 1, 'last_updated_time': 2024-06-19..."
1,final_thomi,benchmarking_result_day_ahead_shop_model_bench...,None,None,bid_configuration_day_ahead_multi_scenario_2_no2,2024-05-01,None,shop_result_benchmarking_dummy_2,True,0.348090,None,"{'space': 'final_thomi', 'external_id': 'Bench...","{'version': 1, 'last_updated_time': 2024-06-19..."
2,final_thomi,benchmarking_result_day_ahead_shop_model_bench...,None,None,bid_configuration_day_ahead_multi_scenario_2_no2,2024-05-02,None,shop_result_benchmarking_dummy_1,True,0.975371,None,"{'space': 'final_thomi', 'external_id': 'Bench...","{'version': 1, 'last_updated_time': 2024-06-19..."
3,final_thomi,benchmarking_result_day_ahead_shop_model_bench...,None,None,bid_configuration_day_ahead_multi_scenario_2_no2,2024-05-02,None,shop_result_benchmarking_dummy_2,True,0.198100,None,"{'space': 'final_thomi', 'external_id': 'Bench...","{'version': 1, 'last_updated_time': 2024-06-19..."
4,final_thomi,benchmarking_result_day_ahead_shop_model_bench...,None,None,bid_configuration_day_ahead_multi_scenario_2_no2,2024-05-03,None,shop_result_benchmarking_dummy_1,True,0.525047,None,"{'space': 'final_thomi', 'external_id': 'Bench...","{'version': 1, 'last_updated_time': 2024-06-19..."
5,final_thomi,benchmarking_result_day_ahead_shop_model_bench...,None,None,bid_configuration_day_ahead_multi_scenario_2_no2,2024-05-03,None,shop_result_benchmarking_dummy_2,True,0.694151,None,"{'space': 'final_thomi', 'external_id': 'Bench...","{'version': 1, 'last_updated_time': 2024-06-19..."
6,final_thomi,benchmarking_result_day_ahead_shop_model_bench...,None,None,bid_configuration_day_ahead_multi_scenario_2_no2,2024-05-04,None,shop_result_benchmarking_dummy_1,True,0.017717,None,"{'space': 'final_thomi', 'external_id': 'Bench...","{'version': 1, 'last_updated_time': 2024-06-19..."
7,final_thomi,benchmarking_result_day_ahead_shop_model_bench...,None,None,bid_configuration_day_ahead_multi_scenario_2_no2,2024-05-04,None,shop_result_benchmarking_dummy_2,True,0.927032,None,"{'space': 'final_thomi', 'external_id': 'Bench...","{'version': 1, 'last_updated_time': 2024-06-19..."
8,final_thomi,benchmarking_result_day_ahead_shop_model_bench...,None,None,bid_configuration_day_ahead_multi_scenario_2_no2,2024-05-05,None,shop_result_benchmarking_dummy_1,True,0.401079,None,"{'space': 'final_thomi', 'external_id': 'Bench...","{'version': 1, 'last_updated_time': 2024-06-19..."
9,final_thomi,benchmarking_result_day_ahead_shop_model_bench...,None,None,bid_configuration_day_ahead_multi_scenario_2_no2,2024-05-05,None,shop_result_benchmarking_dummy_2,True,0.843132,None,"{'space': 'final_thomi', 'external_id': 'Bench...","{'version': 1, 'last_updated_time': 2024-06-19..."


In [10]:
bench_client.shop_result.list()

,space,external_id,case,objective_value,pre_run,post_run,messages,cplex_logs,alerts,output_time_series,node_type,data_record
0,final_thomi,shop_result_benchmarking_dummy_1,shop_case_benchmarking_dummy_1,None,None,None,None,None,None,None,None,"{'version': 1, 'last_updated_time': 2024-06-19..."
1,final_thomi,shop_result_benchmarking_dummy_2,shop_case_benchmarking_dummy_2,None,None,None,None,None,None,None,None,"{'version': 1, 'last_updated_time': 2024-06-19..."


In [11]:
bench_client.benchmarking_configuration_day_ahead.list()

,space,external_id,name,price_area,shop_start_specification,shop_end_specification,bid_configurations,assets_per_shop_model,node_type,data_record
0,final_thomi,day_ahead_benchmarking_configuration_no2,Day Ahead Benchmarking Configuration Dummy,None,[],[],[bid_configuration_day_ahead_multi_scenario_2_...,"[shop_model_to_assets_dummy_1, shop_model_to_a...","{'space': 'final_thomi', 'external_id': 'Bench...","{'version': 1, 'last_updated_time': 2024-06-19..."
